## Atividade Avaliativa 1.2 -- Classificando a Idade do Abalone

O Abalone é um gênero (_Haliotis_) de um moluscos gastrópodes marinhos da família _Haliotidae_. Foi identificado por Linnaeus em 1758 e suas diversas espécies podem ser encontradas em águas costeiras de quase todo o mundo. É usado na indústria alimentícia e em itens decorativos, tais como jóias ou instrumentos musicais [1](https://pt.wikipedia.org/wiki/Abalone). A idade do abalone pode ser obtida diretamente a partir de medidas físicas, porém é necessário cortar a concha, efetuar um processo de pigmentação, e então contar o número de anéis por meio de um microscópio -- tarefa considerada monótona e custosa [2](https://archive.ics.uci.edu/ml/datasets/Abalone).

Outras medidas do Abalone, entretanto, são mais fáceis de obter, não danificam a concha e podem ser utilizadas para estimar a idade com um modelo inteligente por meio de um processo de Aprendizado Supervisionado. Esse é o objetivo do presente projeto prático.

Base de dados original: https://archive.ics.uci.edu/ml/datasets/Abalone  
Base de dados preparada: abalone.csv (Disponível no Google Classroom)

### Bibliotecas

Por hábito, a primeira célula do notebook costuma ser reservada para importação de bibliotecas.
A cada biblioteca nova acrescida, é necessário executar a célula para atualização e correta execução.

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

## Parte 1 - Análise e Exploração de Dados

### Abertura do Dataset
Abra o dataset e visualize o seu cabeçalho, isto é, os primeiros exemplos nele contidos. Isto é útil para checar se a importação foi realizada de maneira adequada e se a disposição dos dados está de acordo para os próximos passos do trabalho.

In [ ]:
df = pd.read_excel(r"C:\Users\STEM - BEX\Documents\GitHub\Redes-Neurais\Projeto prático 1.2\abalone.xlsx", engine="openpyxl")
df.head()


,"Sex,Length,Diameter,Height,Whole weight,Shucked weight,Viscera weight,Shell weight,Age"
0,"M,0.35,0.265,0.09,0.2255,0.0995,0.0485,0.07,7"
1,"F,0.53,0.42,0.135,0.677,0.2565,0.1415,0.21,9"
2,"M,0.44,0.365,0.125,0.516,0.2155,0.114,0.155,10"
3,"I,0.33,0.255,0.08,0.205,0.0895,0.0395,0.055,7"
4,"I,0.425,0.3,0.095,0.3515,0.141,0.0775,0.12,8"


### Conhecendo o dataset

Para praticar conceitos relativos à exploração do conjunto de dados, utilize as células a seguir para prover respostas para cada uma das respostas elencadas

#### Quantos exemplos há no dataset?

In [19]:
print(f'Total de exemplos: {df.shape[0]}')

Total de exemplos: 4176


#### Quais os atributos existentes no dataset e seu tipo?

In [22]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4176 entries, 0 to 4175
Data columns (total 1 columns):
 #   Column                                                                                  Non-Null Count  Dtype 
---  ------                                                                                  --------------  ----- 
 0   Sex,Length,Diameter,Height,Whole weight,Shucked weight,Viscera weight,Shell weight,Age  4176 non-null   object
dtypes: object(1)
memory usage: 32.8+ KB


#### Quantos a idade do abalone mais velho presente na base de dados? Quais seus atributos?

#### Histograma das idades

O histograma é um gráfico que mostra as frequências de uma determinada informação. No nosso caso, vamos construir um histograma que mostra as idades dos abalones. Vamos agrupar as idades em dez bins (caixas).

Ao ler o gráfico, veremos dez barras (bins). Elas sintetizam quantos abalones daquela faixa de idade existem no dataset. Os bins são partições do total de valores em 10 partes de mesmo tamanho.

Documentação completa disponível em: https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.hist.html


#### Boxplot das idades

Como é a distribuição dos valores das idades dos abalones? O gráfico boxplot pode nos ajudar nesta questão.

O boxplot ilustra informações importantes:
- Mediana
- Limite inferior (menor valor)
- Limite superior (maior valor)
- Q1 (primeiro quartil, 25% dos dados)
- Q3 (terceiro quartil, 75% dos dados)
- Outliers (se houver)

A distribuição dos exemplos no dataset é simétrica no tocante à idade? Elabore um parágrafo que justifique as conclusões obtidas a partir da análise do boxplot.

Documentação completa disponível em: https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.boxplot.html

#### Há dados faltantes no dataset?

Dados faltantes (NaN) indicam que durante a coleta e elaboração da base de dados tais informações não estavam disponíveis. Trabalhar com dados faltantes acaba por limitar as nossas análises. Se há dados suficientes, podemos eliminar os dados faltantes. Se há poucos dados, pode-se buscar estratégias para substituir os dados faltantes.

Atividades:

A. Eliminar os dados faltantes.  
B. Quantos exemplos permaneceram no dataset?

#### Sexo do Abalone

Conforme a documentação da base de dados, há três valores possíveis para o sexo do abalone, são elas:  
    - M: Masculino  
    - F: Feminino  
    - I: Infantil (ainda não definido)  
    
Tal como dispostas, segundo strings, essas informações não ajudam modelos que lidam apenas com dados numéricos. É necessário efetuar uma codificação para representar tais valores. Assim, codifique a coluna sexo conforme a abordagem One-Hot Encoding e elimine a coluna original do dataset.

Referências:
- https://towardsdatascience.com/what-is-one-hot-encoding-and-how-to-use-pandas-get-dummies-function-922eb9bd4970
- https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html

## Parte 2 - Tarefa de Aprendizado de Máquina

Abordaremos o problema da previsão de idade como uma tarefa de classificação multiclasse. Embora uma tarefa de regressão pareça mais intuitiva nesse cenário, justificativas práticas motivam tal escolha, tal como a comparação com resultados já existentes na liiteratura.

#### Preparação dos dados para Validação Cruzada Holdout

- Vamos separar o atributo alvo (Age) dos atributores preditores.
  - Atributos preditores devem ser armazenados em uma variável denominada X  
  - Atributo alvo deve ser armazenado em uma variável denominada y
- Qual o tipo de entrada do modelo?
- Vamos efetuar uma partição do tipo holdout 70/30

Documentação completa disponível em: https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html

#### Treinar um modelo de regressão logística para o problema

A Regressão Logística é uma técnica de Aprendizado de Máquina que tem como objetivo produzir, a partir de um conjunto de observações, um modelo que permita a predição de valores tomados por uma variável categórica, frequentemente binária, a partir de uma série de variáveis explicativas.

1. Importe o modelo da biblioteca sklearn
2. Instancie o modelo com parâmetros padrão (default)
3. Execute o algoritmo de treinamento com os dados de treino

Documentação completa disponível em: https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html

#### Obtendo previsões do modelo

1. Selecione um exemplo arbitrário da partição de teste
2. Forneça os atributos preditores e obtenha a previsão da idade
3. O modelo previu a idade corretamente?

#### Avalie o modelo perante o conjunto de testes

1. Obtenha a acurácia, precisão, revocação e F_1 score
  * Efetue a importação de tais métricas a partir do pacote sklearn.metrics
  * Para fins de simplificação, considere os dados balanceados
2. Elabore uma visualização da matriz de confusão para as previsões do conjunto de testes
3. O modelo é bom para o problema? Justifique sua afirmação.

#### Treinar um modelo de K-Vizinhos Mais Próximos

É um algoritmo de Aprendizado de Máquina que baseia-se na média dos valores dos k vizinhos mais próximos a uma dada entrada para produzir uma estimativa para o atributo-alvo. É considerado "preguiçoso", mas muito mostra-se vantajoso por possuir um baixo custo computacional.

1. Efetue o treinamento do K-Vizinhos mais Próximos, considerando k = 5
2. Efetue o teste no modelo e obtenha as métricas de desempenho previamente indicadas.  
3. Comparando com o modelo anterior, qual deles teve melhor desempenho perante o problema? Justifique.

#### Argumente

Qual o grau de dificuldade da tarefa de aprendizado em questão? Justifique a partir dos elementos e resultados obtidos nas células anteriores.